# Calculating cell areas for HEALPIX levels

TODO: replace spherical calculation (here from pyresample) with [spherely](https://github.com/benbovy/spherely)

TODO: generate larger subset of cell per level and compute average cell size

TODO: compare cds-healpix and healpy cell geometries and sizes

## Overview

- generate a cell boundary coordinates (via ipix number) from cds-healpix
- creates a shapely Polygon
- calculates area in different ways: projected centered LAEA, spherical area, "guestimate" subdivision from given Earth's surface by max number of cells in given level

In [2]:
import pyresample
from pyresample.geometry import AreaDefinition
# import pyresample.spherical_geometry
from pyresample import spherical

In [3]:
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import cdshealpix as cds
import astropy.units as u

from shapely.geometry import Polygon, Point

# A=4πr2

# authalic 6371 km
# eqatorial 6378 
radius = 6371

# 510064472 km²
# 510072000
earth_a = 510072000
earth_a_sq = math.pi * 4 * (radius * radius)

print(earth_a , earth_a_sq)
print(earth_a / 12)

def polygons_for_ipix(cell_ids, depth):
    lon, lat = cds.nested.vertices(cell_ids, depth, step=3)
    lon = lon.wrap_at(180 * u.deg)
    polygons = []
    for i, cell_id in enumerate(cell_ids):
        paired_2d = np.column_stack((lon.deg[i], lat.deg[i]))
        points = [ Point(x, y) for (x, y) in paired_2d ]
        poly = Polygon(points)
        polygons.append({'geometry': poly, 'ipix': cell_id})
    return gpd.GeoDataFrame(polygons, crs=4326)

zone_col = []

for l in range(0, 26):
    max_cells = 12 * np.power(4, l)
    
    g = polygons_for_ipix([1], l)
    p = g.geometry[0].centroid

    

    laea_crs_def = f'+proj=laea +lat_0={p.y} +lon_0={p.y} +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs'
    heal_crs_def = '+proj=healpix'
    
    laea_proj = g.to_crs(laea_crs_def)

    heal_proj = g.to_crs(heal_crs_def)

    coords = list(g.geometry[0].exterior.coords)[:-1]
    coords.reverse()
    
    sph_poly = spherical.SphPolygon(np.deg2rad( np.array(coords)), radius=6371)
    
    sphere_area_km2 = sph_poly.area()
    sphere_area_m2 = sphere_area_km2 * 1000000

    laea_projected_area_m2 = laea_proj.geometry[0].area
    laea_projected_area_km2 = laea_projected_area_m2 / 1000000

    heal_projected_area_m2 = heal_proj.geometry[0].area
    heal_projected_area_km2 = heal_projected_area_m2 / 1000000
    
    subdivided_area_km2 = earth_a / max_cells
    subdivided_area_m2 = subdivided_area_km2 * 1000000
    
    zone_col.append({'level': l,
                     'num_cells': max_cells,
                     # 'heal_projected_area_m2' : np.round(heal_projected_area_m2, 4),
                     'laea_projected_area_m2' : np.round(laea_projected_area_m2, 4),
                     'sphere_area_m2' : np.round(sphere_area_m2, 4),
                     'subdivided_area_m2' : np.round(subdivided_area_m2, 4),
                     
                     # 'heal_projected_area_km2': np.round(heal_projected_area_km2, 4),
                     'laea_projected_area_km2': np.round(laea_projected_area_km2, 4),
                     'sphere_area_km2': np.round(sphere_area_km2, 4),
                     'subdivided_area_km2': np.round(subdivided_area_km2, 4),
                     
                    })

pd.options.display.float_format = '{:20,.3f}'.format

level_info = pd.DataFrame(zone_col)
level_info

510072000 510064471.90978825
42506000.0


,level,num_cells,laea_projected_area_m2,sphere_area_m2,subdivided_area_m2,laea_projected_area_km2,sphere_area_km2,subdivided_area_km2
0,0,12,"41,469,891,629,958.164","42,062,053,840,411.023","42,506,000,000,000.000","41,469,891.630","42,062,053.840","42,506,000.000"
1,1,48,"10,565,571,996,934.770","10,581,664,199,360.223","10,626,500,000,000.000","10,565,571.997","10,581,664.199","10,626,500.000"
2,2,192,"2,649,873,227,961.956","2,660,167,722,993.080","2,656,625,000,000.000","2,649,873.228","2,660,167.723","2,656,625.000"
3,3,768,"661,468,605,823.771","664,340,488,270.517","664,156,250,000.000","661,468.606","664,340.488","664,156.250"
4,4,3072,"165,312,492,831.922","166,048,419,803.430","166,039,062,500.000","165,312.493","166,048.420","166,039.062"
5,5,12288,"41,324,801,815.315","41,509,886,436.981","41,509,765,625.000","41,324.802","41,509.886","41,509.766"
6,6,49152,"10,330,994,456.104","10,377,334,018.246","10,377,441,406.250","10,330.995","10,377.334","10,377.441"
7,7,196608,"2,582,735,770.530","2,594,324,921.505","2,594,360,351.562","2,582.736","2,594.325","2,594.360"
8,8,786432,"645,683,140.621","648,580,694.120","648,590,087.891",645.683,648.581,648.590
9,9,3145728,"161,420,735.047","162,145,140.291","162,147,521.973",161.421,162.145,162.148


In [8]:
# !pip install tabulate
# impot tabulate

for l in level_info.map('{:20,.3f}'.format).to_markdown().split("\n"):
    print(l)

|    |   level | num_cells                  | laea_projected_area_m2   | sphere_area_m2         | subdivided_area_m2     | laea_projected_area_km2   | sphere_area_km2   | subdivided_area_km2   |
|---:|--------:|:---------------------------|:-------------------------|:-----------------------|:-----------------------|:--------------------------|:------------------|:----------------------|
|  0 |       0 | 12.000                     | 41,469,891,629,958.164   | 42,062,053,840,411.023 | 42,506,000,000,000.000 | 41,469,891.630            | 42,062,053.840    | 42,506,000.000        |
|  1 |       1 | 48.000                     | 10,565,571,996,934.770   | 10,581,664,199,360.223 | 10,626,500,000,000.000 | 10,565,571.997            | 10,581,664.199    | 10,626,500.000        |
|  2 |       2 | 192.000                    | 2,649,873,227,961.956    | 2,660,167,722,993.080  | 2,656,625,000,000.000  | 2,649,873.228             | 2,660,167.723     | 2,656,625.000         |
|  3 |       3 | 768.000 